In [ ]:
import netCDF4
from netCDF4 import Dataset

import datetime

import pandas as pd
import numpy as np

In [ ]:
"""
Reference Information:

NCEI NetCDF Data Structure Standard:
https://www.ncei.noaa.gov/data/oceans/ncei/formats/netcdf/v2.0/index.html

CF Data Structure Standard:
https://cfconventions.org/Data/cf-conventions/cf-conventions-1.9/cf-conventions.html#profile-data%23profile-data

IOOS Data Standards:
https://ioos.github.io/ioos-metadata/ioos-metadata-profile-v1-2.html#platform


Variable naming conventions:
CF Naming Standards
https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html

Helpful tool to ensure that netCDFs are in compliance with format
https://compliance.ioos.us/index.html
"""

In [ ]:
def make_hydro_ctd_netcdf(filename, data_df, dataqc_df=None, log=None):
    
    """    
    This function takes the filename and sensor data 
    dataframe, and writes it into a netCDF.    
    The structure of the netCDF should be set up in such
    a way that it can be read by an ERDDAP server.
    
    Inputs:
    filename:  string of the filename/filepath
               of the netCDF being created
    data_df:   pandas dataframe containing the data
               to be written into a netCDF
    dataqc_df: pandas dataframe containing the
               QARTOD quality control flags for
               each variable (optional)
    log:       open file to write log messages, 
               if being used (optional)
               
    Outputs:
    None
    """
    
    
    #############################
    # Platform-specific information
    # Change this for your specifc purposes
    
    # Buoy name
    buoy_name = "Se'lhaem - Bellingham Bay"
    
    # Buoy position
    buoy_lat = np.round(48 + (43 + (25.291/60))/60, 8)
    buoy_lon = -np.round(122 + (34 + (35.514/60))/60, 8)
    
    # Buoy depth
    buoy_depth = 25
    
    # WMO Code - for ingestion into NDBC system
    wmo_code = '46118'
    
    # GTS ingestion flag
    gtsflag = 'true'
    
    
    
    #########################################
    # Open up the netCDF file
    dataset = Dataset(filename, 'w')
    
    #
    fillVal = -555

    
    
    #####################################
    # Add netCDF metadata
    #####################################
    
    # The metadata is both helpful as a description
    # of the file itself, the platform collection info
    # and the variables
    dataset.title = "Se'lhaem - Bellingham Bay Surface Hydrological Station Data"
    dataset.description = "Surface Hydrological Data taken from the Se'lhaem - Bellingham Bay buoy"
    dataset.history = "Process historic data on " + datetime.datetime.now().strftime('%Y-%b-%d %H:%M:%S')
    dataset.conventions = 'CF-1.6, ACDD-1.3, IOOS-1.2'
    
    # General metadata fields useful for understanding
    # the datasets, but not necessarily required by
    # and data standard    
    dataset.buoy_designation = buoy_name
    dataset.buoy_latitude = str(np.round(buoy_lat, 6)) +'N'
    dataset.buoy_longitude = str(np.round(buoy_lon, 6)) + 'E'
    dataset.buoy_depth = str(buoy_depth) + ' m'
    dataset.instrument_target_depth = str(target_depth) + ' m'
    dataset.author = 'Seth Travis'
    dataset.contact = 'setht1@uw.edu'
    dataset._FillValue = fillVal
    
    curyear = str(datetime.datetime.now().year)
    dataset.citation = 'Your citation here'
    dataset.institution = 'Northwest Environmental Moorings Group at University of Washington - Applied Physical Laboratory'
    dataset.ror = 'https://ror.org/01a258x16'
    
    
    #   ***   ***   IMPORTANT FIELDS   ***   ***   #
    # These metadata fields are needed by ERDDAP to
    # be able to interpret the datasets
    #   ***   ***   IMPORTANT FIELDS   ***   ***   #
    dataset.cdm_data_type = 'timeSeries'
    dataset.cdm_timeseries_variables = 'location_id, latitude, longitude'
    # Note the "cf_role" metadata attribute for the "location_id" variable
    

    # The global attributes are needed to allow for ingesting by NDBC
    # into the GTS data harvester for models (not necessary)
    dataset.wmo_platform_code = str(wmo_code)
    dataset.gts_ingest = gtsflag

    # These are metadata fields needed for compliance with 
    # NCEI data standards
    dataset.contributor_email = 'setht1@uw.edu, setht1@uw.edu, mickett@uw.edu'
    dataset.contributor_name = 'Seth Travis, Seth Travis, John Mickett'
    dataset.contributor_role = 'author, principalInvestigator, principalInvestigator'
    dataset.contributor_role_vocabulary = 'https://vocab.nerc.ac.uk/collection/G04/current/'
    dataset.contributor_url = 'https://nwem.apl.washington.edu/, https//www.nanoos.org/, https://nwem.apl.washington.edu/'

    dataset.creator_address = 'Henderson Hall, 1013 NE 40th Street, Seattle, WA 98105'
    dataset.creator_city = 'Seattle'
    dataset.creator_country = 'USA'
    dataset.creator_email = 'setht1 at uw.edu'
    dataset.creator_institution = 'Applied Physics Lab - University of Washington (APL-UW)'
    dataset.creator_name = 'Seth Travis'
    dataset.creator_postalcode = '98105'
    dataset.creator_sector = 'academic'
    dataset.create_state = 'WA'
    dataset.creator_type = 'person'
    dataset.creator_url = 'https://nwem.apl.washington.edu/'

    dataset.publisher_address = 'Henderson Hall, 616 NE Northlake Place, Seattle, WA 98105'
    dataset.publisher_city = 'Seattle'
    dataset.publisher_country = 'USA'
    dataset.publisher_email = 'setht1 at uw.edu'
    dataset.publisher_institution = 'Applied Physics Lab - University of Washington (APL-UW)'
    dataset.publisher_name = 'Seth Travis'
    dataset.publisher_postalcode = '98105'
    dataset.publisher_sector = 'academic'
    dataset.publisher_state = 'WA'
    dataset.publisher_type = 'person'
    dataset.publisher_url = 'https://nwem.apl.washington.edu/'

    
    
    #################################################################
    # Create dimensions
    # format is "createDimension('dimension_name',n)"
    # Include descriptive attributes for each dimension
    
    nrows = len(data_df)
    dataset.createDimension('location_id',1)
    dataset.createDimension('time',nrows)
    
    def make_ancvar_str(varname):
        # Define the general QC flag suffixes
        qc_vars = ['qc_agg', 'qc_gross_range_test', 'qc_rate_of_change_test',
                   'qc_spike_test', 'qc_flat_line_test']
        # Initialize an ancillary variable stirng
        ancvar_str = ''
        for qc_var in qc_vars:
            # Add each qc flag specific to the variable to the string
            ancvar_str = ancvar_str + varname + '_' + qc_var + ' '
        # Remove the last ", " from the string
        ancvar_str = ancvar_str[:-1]
        
        return ancvar_str
    
    # Put the variable creation / data writing 
    # into a try statement to ensure that we can
    # close the netCDF if something goes wrong
    try:
        # Create some variables specific to the float. These can be needed
        # for the ERDDAP servers

        # First, create the dimensional variables    
        casttime = dataset.createVariable('time','f8',('location_id','time'))
        casttime.long_name = 'sample_time'
        casttime.description = 'time of sampling'
        casttime.units = 'seconds since 1970-01-01 00:00:00'
        casttime.timezone = 'UTC'
        casttime.calendar = 'gregorian'

        # The variable "buoy_name" is given the cf_role of "timeseries_id",
        # which is needed for CDM data types of "timeseries"
        platform = dataset.createVariable('location_id','S'+str(int(len(buoy_name))),('location_id',))
        platform.long_name = 'location_id'
        platform.description = 'Buoy designation'
        platform.cf_role = 'timeseries_id'
        platform.units = '1'
        platform.ioos_category = 'Identifier'
        platform[0] = buoy_name

        latitude = dataset.createVariable('latitude','f8',('location_id','time'))
        latitude.long_name = 'latitude'
        latitude.description = 'Latitude position in degrees North'
        buoylats = np.array([buoy_lat for ii in range(0,nrows)])
        latitude[:] = buoylats

        longitude = dataset.createVariable('longitude','f8',('location_id','time'))
        longitude.long_name = 'longitude'
        longitude.description = 'Longitude position in degrees East'
        buoylons = np.array([buoy_lon for ii in range(0,nrows)])
        longitude[:] = buoylons
        
        
        instruments = dataset.createVariable('instrument_types', 'str',('location_id','time'))
        instruments.long_name = 'instrument_types'
        instruments.description = 'Names of the instrument(s) used in collecting the sample data'
        instruments[:] = np.expand_dims(data_df['InstrumentType'].to_numpy().squeeze(),axis=0)

        ##############################################
        # Hydrological Sensors
        ##############################################

        # Sea Water Temperature
        temper = dataset.createVariable('sea_water_temperature','f8',('location_id','time'))
        temper.long_name = 'sea_water_temperature'
        temper.description = 'In-situ temperature of water (T90 scale)'
        temper.units = 'degrees C'
        temper.missing_value = fillVal
        temper.gts_ingest = gtsflag
        if dataqc_df is not None:
            temper.ancillary_variables = make_ancvar_str('sea_water_temperature')
        
        # Conductivity
        conduct = dataset.createVariable('sea_water_electrical_conductivity','f8',('location_id','time'))
        conduct.long_name = 'sea_water_electrical_conductivity'
        conduct.description = 'Ability to pass electrical current. In water, it is a proxy from which to derive salinity.'
        conduct.units = 'S/m'
        conduct.missing_value = fillVal
        conduct.gts_ingest = gtsflag
        if dataqc_df is not None:
            conduct.ancillary_variables = make_ancvar_str('sea_water_electrical_conductivity')
        
        # Pressure
        press = dataset.createVariable('sea_water_pressure','f8',('location_id','time'))
        press.long_name = 'sea_water_pressure'
        press.description = 'Pressure exerted by overlying water, excluding air pressure.'
        press.units = 'dbar'
        press.missing_value = fillVal
        press.gts_ingest = gtsflag
        if dataqc_df is not None:
            press.ancillary_variables = make_ancvar_str('sea_water_pressure')
        
        # Depth
        depth = dataset.createVariable('depth','f8',('location_id','time'))
        depth.long_name = 'depth'
        depth.description = 'Z-coordinate of observation in vertical distance below reference. Down is positive. (reference is sea surface)'
        depth.units = 'm'
        depth.missing_value = fillVal
        depth.gts_ingest = gtsflag
        
        # Salinity
        salinity = dataset.createVariable('sea_water_practical_salinity','f8',('location_id','time'))
        salinity.long_name = 'sea_water_practical_salinity'
        salinity.description = 'Salinity is to the salt content of a water sample or body of water. The measure of salt content of a water sample follows UNESCO standards known as the Practical Salinity Scale (PSS) as the conductivity ratio of a sea water sample to a standard KCl solution. PSS is a ratio and has no units.'
        salinity.units = 'psu'
        salinity.missing_value = fillVal
        salinity.reference = 'UNESCO 1983 - Conductivity Ratio to Salinity Conversion' 
        salinity.gts_ingest = gtsflag
        if dataqc_df is not None:
            salinity.ancillary_variables = make_ancvar_str('sea_water_practical_salinity')
        
        # Sea water density (sigma0)
        dens = dataset.createVariable('sea_water_sigma_theta','f8',('location_id','time'))
        dens.long_name = 'sea_water_sigma_theta'
        dens.description = 'Potential density, referenced to 0 dbar, offset by 1000 kg * m^-3 (sigma-0).'
        dens.units = 'kg * m-3 - 1000'
        dens.missing_value = fillVal
        dens.reference = 'UNESCO 1983 - EOS-80 (International Equation of State of Seawater)'
        dens.gts_ingest = gtsflag
        if dataqc_df is not None:
            dens.ancillary_variables = make_ancvar_str('sea_water_sigma_theta')
        
        # Oxygen concentration and fractional saturation
        oxy = dataset.createVariable('mass_concentration_of_oxygen_in_sea_water','f8',('location_id','time'))
        oxy.long_name = 'mass_concentration_of_oxygen_in_sea_water'
        oxy.description = 'Concentration of dissolved oxygen in water'
        oxy.units = 'mg * L^-1'
        oxy.missing_value = fillVal
        oxy.gts_ingest = gtsflag
        if dataqc_df is not None:
            oxy.ancillary_variables = make_ancvar_str('mass_concentration_of_oxygen_in_sea_water')
        
        oxy_sat = dataset.createVariable('fractional_saturation_of_oxygen_in_sea_water','f8',('location_id','time'))
        oxy_sat.long_name = 'fractional_saturation_of_oxygen_in_sea_water'
        oxy_sat.description = 'Concentration of dissolved oxygen in water, as a percentage of the concentration of dissolved oxygen in water at saturation. Dissolved oxygen saturation is the concentration of dissolved oxygen at saturation levels at the same temperature and salinity in a water sample.'
        oxy_sat.units = 'percent (%)'
        oxy_sat.missing_value = fillVal
        oxy_sat.gts_ingest = gtsflag
        if dataqc_df is not None:
            oxy_sat.ancillary_variables = make_ancvar_str('fractional_saturation_of_oxygen_in_sea_water')
        
        # Chlorophyll and Chlorophyll STD
        fluor = dataset.createVariable('mass_concentration_of_chlorophyll_a_in_sea_water','f8',('location_id','time'))
        fluor.long_name = 'mass_concentration_of_chlorophyll_a_in_sea_water'
        fluor.description = 'Mass concentration of chlorophyll-a in sea water. Measurement is based upon fluoroescence taken from fluorometer'
        fluor.units = 'mg * m-3'
        fluor.missing_value = fillVal
        fluor.gts_ingest = gtsflag
        if dataqc_df is not None:
            fluor.ancillary_variables = make_ancvar_str('mass_concentration_of_chlorophyll_a_in_sea_water')
        
        fluor_std = dataset.createVariable('mass_concentration_of_chlorophyll_a_in_sea_water_std','f8',('location_id','time'))
        fluor_std.long_name = 'mass_concentration_of_chlorophyll_a_in_sea_water_std'
        fluor_std.description = 'Standard deviation of mass concentration of chlorophyll-a in sea water. Measurement is based upon fluoroescence taken from fluorometer'
        fluor_std.units = 'mg * m-3'
        fluor_std.missing_value = fillVal
        fluor_std.gts_ingest = 'false'
        
        # Turbidity and Turbidity STD
        turbid = dataset.createVariable('sea_water_turbidity','f8',('location_id','time'))
        turbid.long_name = 'sea_water_turbidity'
        turbid.description = 'Measure of light scattering due to suspended material in water. Measure of the cloudiness of water. Higher turbidity levels are often associated with higher levels of disease-causing microorganisms such as viruses, parasites and some bacteria. Turbidity is measured in nephelometric turbidity units (NTU)'
        turbid.units = 'NTU'
        turbid.missing_value = fillVal
        turbid.gts_ingest = gtsflag
        if dataqc_df is not None:
            turbid.ancillary_variables = make_ancvar_str('sea_water_turbidity')
        
        turbid_std = dataset.createVariable('sea_water_turbidity_std','f8',('location_id','time'))
        turbid_std.long_name = 'sea_water_turbidity_std'
        turbid_std.description = 'Standard deviation of measure of light scattering due to suspended material in water. Measure of the cloudiness of water. Higher turbidity levels are often associated with higher levels of disease-causing microorganisms such as viruses, parasites and some bacteria. Turbidity is measured in nephelometric turbidity units (NTU)'
        turbid_std.units = 'NTU'
        turbid_std.missing_value = fillVal
        turbid_std.gts_ingest = 'false'
        
        # pH
        pH = dataset.createVariable('sea_water_ph_reported_on_total_scale','f8',('location_id','time'))
        pH.long_name = 'sea_water_ph_reported_on_total_scale'
        pH.description = 'measure of acidity of seawater, defined as the negative logarithm of the concentration of dissolved hydrogen ions'
        pH.units = '1'
        pH.missing_value = fillVal
        pH.gts_ingest = 'false'
        
        
        ###########################################################################
        # Assign the relevant data into the netCDF variables

        # Apply the date offset for the cast times
        casttime[:] = data_df['time']

        # Physical parameters
        temper[:] = data_df['sea_water_temperature']
        conduct[:] = data_df['sea_water_electrical_conductivity']
        press[:] = data_df['sea_water_pressure']
        depth[:] = data_df['depth']
        salinity[:] = data_df['sea_water_practical_salinity']
        dens[:] = data_df['sea_water_sigma_theta']

        # Chemical parameters
        oxy[:] = data_df['mass_concentration_of_oxygen_in_sea_water']
        oxy_sat[:] = data_df['fractional_saturation_of_oxygen_in_sea_water']
        fluor[:] = data_df['mass_concentration_of_chlorophyll_a_in_sea_water']
        fluor_std[:] = data_df['mass_concentration_of_chlorophyll_a_in_sea_water_std']
        turbid[:] = data_df['sea_water_turbidity']
        turbid_std[:] = data_df['sea_water_turbidity_std']
        pH[:] = data_df['sea_water_ph_reported_on_total_scale']
        
        
        
        ##############################################
        # QARTOD Data
        # Quality Control Flagging Data
        
        if dataqc_df is not None:
        
            varnames = ['sea_water_temperature',
                        'sea_water_electrical_conductivity',
                        'sea_water_pressure',
                        'sea_water_practical_salinity',
                        'sea_water_sigma_theta',
                        'mass_concentration_of_oxygen_in_sea_water',
                        'mass_concentration_of_chlorophyll_a_in_sea_water',
                        'sea_water_turbidity',
                        'sea_water_ph_reported_on_total_scale']
            varlabs = ['Sea Water Temperature', 
                       'Sea Water Electrical Conductivity', 
                       'Sea Water Pressure', 
                       'Sea Water Practical Salinity',
                       'Sea Water Sigma Theta',
                       'Mass Concentration of Oxygen in Sea Water', 
                       'Mass Concentration of Chlorophyll_a in Sea Water', 
                       'Sea Water Turbidity',
                       'Sea Water pH']

            origqc_vars = ['qartod_rollup_qc', 
                           'qartod_gross_range_test', 'qartod_rate_of_change_test',
                           'qartod_spike_test', 'qartod_flat_line_test',
                           'qartod_manually_flagged_test']
            qc_vars = ['qc_agg', 
                       'qc_gross_range_test', 'qc_rate_of_change_test',
                       'qc_spike_test', 'qc_flat_line_test',
                       'qc_manually_flagged_test']
            qc_standards = ['aggregate_quality_flag', 
                            'gross_range_test_quality_flag', 
                            'rate_of_change_test_quality_flag', 'spike_test_quality_flag',
                            'flat_line_test_quality_flag',
                            'manually_flagged_test']
            qc_labs = ['Aggregate Flag', 'Gross Range Test Flag', 
                       'Rate of Change Test Flag', 'Spike Test Flag',
                       'Flat Line Test Flag',
                       'Manually Flagged Test']

            for ii in range(0,len(varnames)):
                # Step through each QARTOD test for each variable, and write it to the file
                var = varnames[ii]
                varlabel = varlabs[ii]
                for jj in range(0,len(qc_vars)):
                    origqc_var = origqc_vars[jj]
                    qc_var = qc_vars[jj]
                    qc_standard = qc_standards[jj]
                    qc_lab = qc_labs[jj]

                    # Create new variable
                    temp_qcvar = dataset.createVariable(var + '_' + qc_var,'i4',('location_id','time'))
                    temp_qcvar.standard_name = qc_standard
                    temp_qcvar.long_name = var + '_' + qc_var
                    temp_qcvar.description = varlabel + ' ' + qc_lab
                    temp_qcvar.ioos_category = 'Quality Control'
                    temp_qcvar.units = '1'
                    temp_qcvar.coverage_content_type = 'qualityInformation'
                    temp_qcvar.flag_vals = '1, 2, 3, 4, 9'
                    temp_qcvar.flag_meanings = 'PASS NOT_EVALUATED SUSPECT FAIL MISSING'
                    temp_qcvar[:] = dataqc_df[var + '_' + origqc_var]
        
    except Exception as e:
        print('Error occurred when making the netCDF file', file=log)
        print(e, file=log)
    
    # Be sure to close the file before ending the function
    dataset.close()
    
    
    return 